In [ ]:
!pip3 install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip3 install -q pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

In [ ]:
!pip install -q pyg-lib git+https://github.com/pyg-team/pyg-lib.git torch-scatter torch-sparse torch-geometric

You don't need to run the pip installs that are above.

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from PIL import Image

In [2]:
pixel_dict = {}

In [3]:
#file = "benchmark_data/C4_flickr/tree.png"
#file = "benchmark_data/C4_flickr/tree_copy.png"
#file = "benchmark_data/C4_flickr/giraffe.png"
file = "benchmark_data/C4_flickr/giraffe_copy.png"

In [4]:
dict_file = open(file+".pkl", 'rb')
pixel_dict = pickle.load(dict_file)
dict_file.close()

Since this was primarly focused on exploing the relationship between these pixels we have two seperate dictionaries. normalized_dict is where we store the averaged pixels. edited_dict stores the origional pixel subtracted by the averaged pixel color.

In [5]:
normalized_dict = {}

In [6]:
edited_dict = {}

In this loop we find the average of the pixels. This is very similar to how a node can be predicted by its neighbors and part of where the inspiration came from. The results are stored in the dictionaries above.

In [7]:
for key in pixel_dict:
    pixel = tuple(np.divide(sum(np.array(t) for t in pixel_dict[key]), len(pixel_dict[key])))
    
    tmp = tuple(map(round, pixel))
    
    edited_dict[key] = tuple(map(round, tuple(map(lambda p: max(0, p), np.subtract(key, pixel)))))
    normalized_dict[tmp] = normalized_dict.get(tmp, []) + [key]
    

In the cells below we are looking at how many lists of each length there are. Fundimentally if an image is resuing exact elements there will longer lengths. One thing to watch out for is landscape images with large open spaces. 

In [8]:
dist = {}
for key in normalized_dict:
    tmp = normalized_dict[key]
    dist[len(tmp)] = dist.get(len(tmp), 0) + 1

In [9]:
print(dist, max(dist))

{1: 104669, 2: 8921, 3: 686, 4: 74, 5: 7, 7: 1} 7


In [10]:
dist2 = {}
for key in edited_dict:
    tmp = edited_dict[key]
    dist2[len(tmp)] = dist2.get(len(tmp), 0) + 1

In [11]:
print(dist2, max(dist2))

{3: 124907} 3


In this cell block we are creating a blacklist of pixel colors. This list is used to change the color of specific pixels later.

In [12]:
count = 0
l = set()
for key in normalized_dict:
    tmp = normalized_dict[key]
    #5
    if len(tmp) > 5:
        l.update(tmp)
    else: 
        count += 1
        
print(count)

114357


In [13]:
print("Pixels: ", len(l))

Pixels:  7


In [14]:
img = Image.open(file)

In [15]:
width, height = img.size

In this loop we update the picture to include the detection. For easy viewing I have specifically choosen a bright orange color, however it is super easy to change. With this setup we are able to see places where editing likely took place.

If we uncomment out the img.putpixel we can normalize the entire photo. This removes some details if we are not careful.

In [16]:
for x in range(width):
    for y in range(height):
        #img.putpixel((x, y), normalized_dict[img.getpixel((x, y))])
        # or
        if img.getpixel((x, y)) in l:
            img.putpixel((x, y), (240, 74, 0))

Intrestingly if we edit the loop to be the following we can get a ruff boundary detection system. If using the cell above skip this cell.

In [ ]:
for x in range(width):
    for y in range(height):
        if (edited_dict[img.getpixel((x, y))]) == (0,0,0):
            img.putpixel((x, y), edited_dict[img.getpixel((x, y))])
        else:
            img.putpixel((x, y), (240, 74, 0))

In [17]:
img.save(file+".png")